# MNIST Project
> **By Jisang Yun**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision 
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
transform = transforms.Compose([
    transforms.RandomRotation(degrees=15),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [ ]:
train_dataset = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=transform,
)

test_dataset = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=transform,
)

In [ ]:
print('Train Size: ' + str(len(train_dataset)) + ', Test Size: ' + str(len(test_dataset)))

In [ ]:
train_dataset[1]

In [ ]:
batch_size = 64
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

In [ ]:
for example_data, example_labels in train_loader:
    example_image = example_data[0]
    print("Input Size:", example_data.size())
    
    example_image_numpy = example_image.permute(1, 2, 0).numpy()  # Convert to HWC format for visualization

    plt.imshow(example_image_numpy)
    plt.title(f"Label: {example_labels[0]}")
    plt.show()

    break

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

        if in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1),
                nn.BatchNorm2d(out_channels)
            )
        else:
            self.shortcut = nn.Sequential()

    def forward(self, x):
        residual = self.shortcut(x)

        # Apply first convolution, batch normalization, and ReLU
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        # Apply second convolution and batch normalization
        out = self.conv2(out)
        out = self.bn2(out)

        # Skip connection
        out += residual
        out = self.relu(out)
        
        return out

In [ ]:
class SimpleCNN(nn.Module):
   def __init__(self):
       super(SimpleCNN, self).__init__()
    #    # Convolutional layers
    #    self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
    #    self.bn1 = nn.BatchNorm2d(32)  # Batch normalization after conv1

    #    self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
    #    self.bn2 = nn.BatchNorm2d(64)

    #    self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
    #    self.bn3 = nn.BatchNorm2d(128)
       
    #    # Activation and pooling layers
    #    self.relu = nn.ReLU()

       # Residual blocks
       self.res_block1 = ResidualBlock(1, 32)
       self.res_block2 = ResidualBlock(32, 64)
       self.res_block3 = ResidualBlock(64, 128)

       self.pool = nn.MaxPool2d(kernel_size=2, stride=2)  # Only define once
       self.relu = nn.ReLU()
       
       # Automatically calculate the input size for fully connected layer
       self.fc_input_size = self._get_conv_output_size()
       
       # Fully connected layers
       self.fc1 = nn.Linear(self.fc_input_size, 128)
       self.dropout = nn.Dropout(0.5)
       self.fc2 = nn.Linear(128, 20)
       self.fc3 = nn.Linear(20, 10)

   def _get_conv_output_size(self):
       """Automatically calculate the output size of convolutional layers"""
       with torch.no_grad():
           # Test with MNIST image size (1, 28, 28)
           x = torch.zeros(1, 1, 28, 28)
           
           # Forward pass through conv layers
        #    x = self.pool(self.relu(self.conv1(x)))  # 28x28 -> 14x14
        #    x = self.pool(self.relu(self.conv2(x)))  # 14x14 -> 7x7
        #    x = self.pool(self.relu(self.conv3(x)))  # 7x7 -> 3x3

           # Forward pass through residual blocks
           x = self.pool(self.res_block1(x))  # 28x28 -> 14x14
           x = self.pool(self.res_block2(x))  # 14x14 -> 7x7
           x = self.pool(self.res_block3(x))  # 7x7 -> 3x3
           
           return x.view(1, -1).size(1)

   def forward(self, x):
       # Convolutional layers with ReLU and pooling
    #    x = self.pool(self.relu(self.conv1(x)))  # (batch, 32, 14, 14)
    #    x = self.pool(self.relu(self.conv2(x)))  # (batch, 64, 7, 7)
    #    x = self.pool(self.relu(self.conv3(x)))  # (batch, 128, 3, 3)
       
       # Residual blocks with pooling
       x = self.pool(self.res_block1(x))  # (batch, 32, 14, 14)
       x = self.pool(self.res_block2(x))  # (batch, 64, 7, 7)
       x = self.pool(self.res_block3(x))  # (batch, 128, 3, 3)

       # Flatten the tensor for fully connected layers
       x = x.view(x.size(0), -1)  # Safer and simpler method
       
       # Fully connected layers
       x = self.relu(self.fc1(x))
       x = self.dropout(x)
       x = self.relu(self.fc2(x))  # Add ReLU to fc2 as well
       x = self.fc3(x)  # Final output layer (no activation)
       
       return x


In [ ]:
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Lists to track training loss
train_losses = []
epoch_losses = []

In [ ]:
num_epochs = 50
running_loss = 0.0

for epoch in range(num_epochs):
    epoch_loss = 0.0
    batch_count = 0

    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs.float())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Save current batch loss
        current_loss = loss.item()
        train_losses.append(current_loss)

        running_loss += loss.item()
        if i % 100 == 99:
            print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 100:.3f}")
            running_loss = 0.0
        
    # Calculate and store epoch average loss
    avg_epoch_loss = epoch_loss / batch_count if batch_count > 0 else 0
    epoch_losses.append(avg_epoch_loss)

print('Finished Training')

In [ ]:
# Visualize training loss
plt.figure(figsize=(12, 4))

# Plot batch-wise loss
plt.subplot(1, 2, 1)
plt.plot(train_losses, alpha=0.7, linewidth=0.8)
plt.title('Training Loss per Batch')
plt.xlabel('Batch Number')
plt.ylabel('Loss')
plt.grid(True, alpha=0.3)

# Plot epoch-wise average loss
plt.subplot(1, 2, 2)
plt.plot(range(1, num_epochs + 1), epoch_losses, 'b-', linewidth=2, marker='o', markersize=3)
plt.title('Average Training Loss per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Average Loss')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Print loss statistics
print(f"\nInitial Loss: {train_losses[0]:.4f}")
print(f"Final Loss: {train_losses[-1]:.4f}")
print(f"Best Epoch Loss: {min(epoch_losses):.4f}")
print(f"Loss Reduction: {train_losses[0] - train_losses[-1]:.4f}")

In [ ]:
model.eval()
predictions = []

with torch.no_grad():
    for data in test_loader:
        data = data.to(device)

        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(predicted.cpu().tolist())

submission = pd.DataFrame({
    "ImageId": range(1, len(predictions) + 1),
    "Label": predictions
})

submission.to_csv('submission.csv', index=False)